In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geoplot as gplt
import geojson
from geojson import Feature, FeatureCollection, Point
import json
import fiona; fiona.supported_drivers

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'raw',
 'DXF': 'raw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRI Shapefile': 'raw',
 'GeoJSON': 'rw',
 'GPKG': 'rw',
 'GML': 'raw',
 'GPX': 'raw',
 'GPSTrackMaker': 'raw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

# Load Data

In [2]:
acq_colms=['LoanIdentifier','SellerName', 'OriginalInterestRate', 
           'OriginalUPB', 'OriginalLoanTerm', 'OriginationDate', 
           'FirstPaymentDate', 'OriginalLoanToValueLTV', 'PrimaryMortgageInsurancePercent', 
           'OriginalDebtToIncomeRatio', 'NumberofBorrowers', 'FirstTimeHomeBuyerIndicator', 
           'BorrowerCreditScoreAtOrigination', 'PropertyState', 'ZipCodeShort', 
           'CoBorrowerCreditScoreAtOrigination'];

per_colms= ["LoanIdentifier","MonthlyReportingPeriod","LoanAge","RemainingMonthstoMaturity","AdjustedMonthstoMaturity",
            "MaturityDate", "MetropolitanStatisticalAreaMSA","CurrentLoanDelinquencyStatus",
            "ForeclosureDate"];

In [3]:
acq_data= pd.read_csv('all_acq_data.txt', sep='|', header=None, names=acq_colms)

per_data= pd.read_csv('all_Per_Data.txt', sep='|', header=None, names=per_colms)

In [4]:
acq_data.head()

,LoanIdentifier,SellerName,OriginalInterestRate,OriginalUPB,OriginalLoanTerm,OriginationDate,FirstPaymentDate,OriginalLoanToValueLTV,PrimaryMortgageInsurancePercent,OriginalDebtToIncomeRatio,NumberofBorrowers,FirstTimeHomeBuyerIndicator,BorrowerCreditScoreAtOrigination,PropertyState,ZipCodeShort,CoBorrowerCreditScoreAtOrigination
0,100004457300,"WELLS FARGO BANK, N.A.",3.875,304000,360,11/2014,01/2015,80,0,42.0,1,0,813.0,GA,300,0.0
1,100020572903,OTHER,4.250,348000,360,01/2015,03/2015,70,0,42.0,1,1,726.0,TX,787,0.0
2,100040063822,OTHER,4.375,521000,360,12/2014,02/2015,90,25,23.0,2,1,783.0,VA,220,807.0
3,100047753781,QUICKEN LOANS INC.,4.875,89000,360,02/2015,04/2015,95,30,38.0,1,1,715.0,PA,190,0.0
4,100051904183,OTHER,4.000,261000,360,02/2015,04/2015,95,30,36.0,2,0,662.0,KY,400,767.0


In [5]:
per_data.head()

,LoanIdentifier,MonthlyReportingPeriod,LoanAge,RemainingMonthstoMaturity,AdjustedMonthstoMaturity,MaturityDate,MetropolitanStatisticalAreaMSA,CurrentLoanDelinquencyStatus,ForeclosureDate
0,100004457300,01/01/2015,1,359,359,12/2044,12060,0,0
1,100004457300,02/01/2015,2,358,357,12/2044,12060,0,0
2,100004457300,03/01/2015,3,357,356,12/2044,12060,0,0
3,100004457300,04/01/2015,4,356,355,12/2044,12060,0,0
4,100004457300,05/01/2015,5,355,354,12/2044,12060,0,0


### Merge Data Sets

In [6]:
per_data.drop_duplicates(subset='LoanIdentifier', keep='last', inplace=True)
mergedData= pd.merge(acq_data, per_data, how='left')

mergedData.rename(index=str, columns={'ForeclosureDate': 'Foreclosure'}, inplace=True)

In [7]:
pd.options.display.max_columns= 200
mergedData.head()

,LoanIdentifier,SellerName,OriginalInterestRate,OriginalUPB,OriginalLoanTerm,OriginationDate,FirstPaymentDate,OriginalLoanToValueLTV,PrimaryMortgageInsurancePercent,OriginalDebtToIncomeRatio,NumberofBorrowers,FirstTimeHomeBuyerIndicator,BorrowerCreditScoreAtOrigination,PropertyState,ZipCodeShort,CoBorrowerCreditScoreAtOrigination,MonthlyReportingPeriod,LoanAge,RemainingMonthstoMaturity,AdjustedMonthstoMaturity,MaturityDate,MetropolitanStatisticalAreaMSA,CurrentLoanDelinquencyStatus,Foreclosure
0,100004457300,"WELLS FARGO BANK, N.A.",3.875,304000,360,11/2014,01/2015,80,0,42.0,1,0,813.0,GA,300,0.0,06/01/2019,54,306,290,12/2044,12060,0,0
1,100020572903,OTHER,4.250,348000,360,01/2015,03/2015,70,0,42.0,1,1,726.0,TX,787,0.0,09/01/2016,19,341,0,02/2045,12420,X,0
2,100040063822,OTHER,4.375,521000,360,12/2014,02/2015,90,25,23.0,2,1,783.0,VA,220,807.0,11/01/2015,10,350,0,01/2045,47900,X,0
3,100047753781,QUICKEN LOANS INC.,4.875,89000,360,02/2015,04/2015,95,30,38.0,1,1,715.0,PA,190,0.0,06/01/2019,51,309,309,03/2045,37980,0,0
4,100051904183,OTHER,4.000,261000,360,02/2015,04/2015,95,30,36.0,2,0,662.0,KY,400,767.0,05/01/2019,50,310,0,03/2045,31140,X,0


In [8]:
mapdf= pd.DataFrame(mergedData, columns={'LoanIdentifier':'LoanIdentifier', 'OriginalInterestRate': 'OriginalInterestRate',
                    'OriginalUPB':'OriginalUPB', 'FirstTimeHomeBuyerIndicator': 'FirstTimeHomeBuyerIndicator',
                    'BorrowerCreditScoreAtOrigination':'BorrowerCreditScoreAtOrigination', 'PropertyState':'PropertyState',
                    'Foreclosure':'Foreclosure'})
mapdf.head()

,LoanIdentifier,OriginalInterestRate,OriginalUPB,FirstTimeHomeBuyerIndicator,BorrowerCreditScoreAtOrigination,PropertyState,Foreclosure
0,100004457300,3.875,304000,0,813.0,GA,0
1,100020572903,4.250,348000,1,726.0,TX,0
2,100040063822,4.375,521000,1,783.0,VA,0
3,100047753781,4.875,89000,1,715.0,PA,0
4,100051904183,4.000,261000,0,662.0,KY,0


In [9]:
mapdf.dtypes

LoanIdentifier                        int64
OriginalInterestRate                float64
OriginalUPB                           int64
FirstTimeHomeBuyerIndicator           int64
BorrowerCreditScoreAtOrigination    float64
PropertyState                        object
Foreclosure                           int64
dtype: object

In [15]:
number=(mapdf['Foreclosure']==1).sum()
number

1215

In [16]:
#Quick Look at Values per State
mapdf['PropertyState'].value_counts()

CA    168997
TX    104413
FL     85411
IL     82147
PA     79478
NY     76767
MI     75807
WI     71060
OH     68354
MN     67769
WA     62477
NJ     51151
MA     45518
NC     42643
GA     41195
OR     39461
CO     38521
IN     37925
TN     37522
VA     35782
MO     34732
IA     34726
UT     34710
AZ     32716
MD     28254
CT     25920
LA     22853
AL     21771
KY     21436
NE     20800
SC     19530
ID     17028
OK     16481
KS     14114
AR     13971
NM     12709
NV     12693
SD     11013
NH     10946
MS      9779
MT      9047
RI      7571
ND      6532
ME      6038
WY      5555
WV      4874
DE      4203
AK      3439
VT      3144
DC      2552
HI      2333
PR      2196
VI        79
GU        59
Name: PropertyState, dtype: int64

In [17]:
#Change State from Object to Categorical
mapdf['PropertyState']=pd.Categorical(mapdf['PropertyState'], ordered= True)
mapdf.dtypes

LoanIdentifier                         int64
OriginalInterestRate                 float64
OriginalUPB                            int64
FirstTimeHomeBuyerIndicator            int64
BorrowerCreditScoreAtOrigination     float64
PropertyState                       category
Foreclosure                            int64
dtype: object

In [18]:
# Summarize mapdf Table by State
#pd.set_option('precision',0)
avg_InterestRate= mapdf.groupby('PropertyState')['OriginalInterestRate'].mean().round(3)
min_Interest= mapdf.groupby('PropertyState')['OriginalInterestRate'].min().round(3)
max_Interest= mapdf.groupby('PropertyState')['OriginalInterestRate'].max().round(3)

avg_UPB= mapdf.groupby('PropertyState')['OriginalUPB'].mean().round(0)
avg_CreditScore= mapdf.groupby('PropertyState')['BorrowerCreditScoreAtOrigination'].mean().round(0)
sum_FirstTimeHomeBuyers= mapdf.groupby('PropertyState')['FirstTimeHomeBuyerIndicator'].sum()
sum_Foreclosures= mapdf.groupby('PropertyState')['Foreclosure'].sum()

#New Summarized DF
StateInfo= pd.DataFrame({"AverageInterestRate":avg_InterestRate, "MinInterestRate":min_Interest, 
                         "MaxInterestRate":max_Interest, "AverageUPB":avg_UPB, "AverageCreditScore":avg_CreditScore,
                        "NumberFirstTimeHomeBuyers":sum_FirstTimeHomeBuyers, "NumberOfForeclosures":sum_Foreclosures})
StateInfo.reset_index()

#Remove unwanted decimal places
StateInfo["AverageUPB"]=StateInfo["AverageUPB"].map(int)
StateInfo["AverageCreditScore"]=StateInfo["AverageCreditScore"].map(int)
StateInfo

,AverageInterestRate,MinInterestRate,MaxInterestRate,AverageUPB,AverageCreditScore,NumberFirstTimeHomeBuyers,NumberOfForeclosures
PropertyState,,,,,,,
AK,3.995,2.375,5.875,282063,758,1320,5
AL,4.076,2.125,6.625,173832,751,9100,44
AR,4.033,2.375,6.625,160289,753,5293,19
AZ,4.400,2.375,6.500,219479,743,17088,23
CA,4.220,2.375,6.625,358495,750,87672,62
CO,4.202,2.250,6.500,284203,756,18621,18
CT,3.941,2.250,6.125,248991,753,15268,14
DC,4.172,2.375,5.875,456502,760,1683,0
DE,4.166,2.375,6.625,216887,752,2348,3


In [19]:
StateInfo['NumberOfForeclosures'].sum()

1215

In [13]:
#Save File to Merge with map
#StateInfo.to_csv('states.csv', index=False, header=True)
StateInfo.to_json('states.json', orient='table')

# If Using geopandas

### Load GeoJson

In [14]:
#Load GeoJson
geoFrame= 'https://www1.ncdc.noaa.gov/pub/data/nidis/geojson/us/base/unitedstates.geojson'

df = gpd.read_file(geoFrame)
print(type(df))

#Rename State to PropertyState
statedf = df.rename(columns={'STATE':'PropertyState'}) 
statedf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,PropertyState,geometry
0,AK,"MULTIPOLYGON (((-179.10000 51.23000, -179.1300..."
1,AL,"MULTIPOLYGON (((-88.08000 30.25000, -88.09000 ..."
2,AR,"POLYGON ((-94.36000 36.50000, -94.08000 36.500..."
3,AZ,"POLYGON ((-110.50000 37.00000, -110.47000 37.0..."
4,CA,"MULTIPOLYGON (((-118.37000 32.84000, -118.3500..."


In [15]:
statedf.dtypes

PropertyState      object
geometry         geometry
dtype: object

In [16]:
statedf['PropertyState']=pd.Categorical(statedf['PropertyState'], ordered= True)
statedf.dtypes

PropertyState    category
geometry         geometry
dtype: object

# Merge dfs

In [17]:
#Merged Pandas df
df= pd.merge(StateInfo, statedf,on='PropertyState', how='inner')
df
#The Reason for the difference in rows is because there is no GeoJSON information for GU=Guam available

,PropertyState,AverageInterestRate,MinInterestRate,MaxInterestRate,AverageUPB,AverageCreditScore,NumberFirstTimeHomeBuyers,NumberOfForeclosures,geometry
0,AK,3.995,2.375,5.875,282063,758,1320,5,"MULTIPOLYGON (((-179.10000 51.23000, -179.1300..."
1,AL,4.076,2.125,6.625,173832,751,9100,44,"MULTIPOLYGON (((-88.08000 30.25000, -88.09000 ..."
2,AR,4.033,2.375,6.625,160289,753,5293,19,"POLYGON ((-94.36000 36.50000, -94.08000 36.500..."
3,AZ,4.400,2.375,6.500,219479,743,17088,23,"POLYGON ((-110.50000 37.00000, -110.47000 37.0..."
4,CA,4.220,2.375,6.625,358495,750,87672,62,"MULTIPOLYGON (((-118.37000 32.84000, -118.3500..."
5,CO,4.202,2.250,6.500,284203,756,18621,18,"POLYGON ((-106.45000 41.00000, -106.32000 41.0..."
6,CT,3.941,2.250,6.125,248991,753,15268,14,"POLYGON ((-72.81000 42.04000, -72.82000 42.000..."
7,DC,4.172,2.375,5.875,456502,760,1683,0,"POLYGON ((-77.12000 38.93000, -77.04000 39.000..."
8,DE,4.166,2.375,6.625,216887,752,2348,3,"MULTIPOLYGON (((-75.56000 39.63000, -75.56000 ..."
9,FL,4.300,2.250,6.625,205651,743,45135,32,"MULTIPOLYGON (((-81.91000 24.53000, -81.97000 ..."


In [18]:
print(type(df))
df.dtypes

<class 'pandas.core.frame.DataFrame'>


PropertyState                category
AverageInterestRate           float64
MinInterestRate               float64
MaxInterestRate               float64
AverageUPB                      int64
AverageCreditScore              int64
NumberFirstTimeHomeBuyers       int64
NumberOfForeclosures            int64
geometry                     geometry
dtype: object

In [19]:
#Convert df to GeoPandas df
df_gdf= gpd.GeoDataFrame(df, geometry='geometry')
print(type(df))

<class 'pandas.core.frame.DataFrame'>
